---
layout: post
title: Stereo vision and disparity maps (in Julia)
categories: [julia, coding, stereo]
excerpt: An introduction into basic stereo vision, with a simple block matching algorithm written in Julia. 
---

I've been working a lot recently with stereo vision and wanted to go through the basics of how disparity is calculated. I'm partially doing this as an excuse to get better at Julia (v1.9.3).  

## Introduction

In much the same way that we as humans can have depth perception by sensing the difference in the images we see between our left and right eyes, we can calculate depth from a pair of images taken from different locations, called a stereo pair.   

If we know the positions of out cameras, then we can use matching points in our two images to estimate how far away from the camera those points are. 

Taking a look at the image below (from [OpenCV](https://docs.opencv.org/4.x/dd/d53/tutorial_py_depthmap.html)):

![https://docs.opencv.org/4.x/dd/d53/tutorial_py_depthmap.html](../images/dispairity_block_julia/stereo_depth.jpg)

If we have two identical cameras, at points $$O$$ and $$O'$$ at a distance $$B$$ from each other, with focal length $$f$$, we can calculate the distance ($$Z$$) to object $$X$$ by using the *disparity* between where the object $$X$$ appears in the *left* image ($$x$$) and where it appears in the *right* image ($$x'$$).  

In this simple case, the relation between disparity and distance is simply:

\begin{equation}
disparity = x - x' = \frac{Bf}{Z}
\end{equation}

If we know $$B$$ an $$f$$, then we can rearrange this to give us distance as a function of disparity: 

\begin{equation}
Z = \frac{Bf}{x - x'}
\end{equation}

You might notice that in case the disparity is zero, you will have an undefined result. This is just due to the fact that in this case the cameras are pointing in parallel, so in principle a disparity of zero should not be possible.   

The general case is more complicated, but we will focus on this simple setup for now.  

We can define the function as: 

In [ ]:
function distance_from_disparity(B, f, disparity)
    B*f/disparity
end

Where $$B$$ and $$disparity$$ are measured in pixels, and $$f$$ is measured in centimeters. 

There is an inverse relation between distance and disparity: 

In [ ]:
using Plots

disparities = range(1, 50, length=50)
distances = distance_from_disparity.(100, 0.1, disparities)

plot(disparities, distances, label="Distance [cm]")
xlabel!("Disparity")
ylabel!("Distance [cm]")

So once we have a disparity, it's relatively straightforward to get a distance. But how do we find disparities? 

## Disparity maps

We usually represent the disparities for a given pair of images as a *disparity map*, which is an array with the same dimensions as (one of) your images, but with disparity values for each pixel. 

In principle, this is a two-dimensional problem, as an object might be matched to a point that has both a horizontal and vertical shift, but luckily, you can always find a transformation to turn this into a one dimensional problem.  

The cartoon below illustrates what a disparity map might look like: 
![Own work](../images/dispairity_block_julia/disparity_cartoon.png)

Above, we calculate the disparity with respect to the left image (you can do it with respect to the right image as well), and as you can see the disparity map tells us how many pixels to the right each object shifted to the left image vs the right image.  

For a set of images (taken from the [Middlebury Stereo Datasets](https://vision.middlebury.edu/stereo/data/)):  
![https://vision.middlebury.edu/stereo/eval/newEval/tsukuba/](../images/dispairity_block_julia/im3.png) ![https://vision.middlebury.edu/stereo/eval/newEval/tsukuba/](../images/dispairity_block_julia/im4.png)   

The corresponding disparity map can be visualized as follows:   

![https://vision.middlebury.edu/stereo/eval/newEval/tsukuba/](../images/dispairity_block_julia/groundtruth.png) 

With darker pixels having lower disparity values, and brighter pixels having higher disparity values, meaning the dark objects are far away from the cameras, while the bright ones are close.   

The ground truth disparity as shown above is usually calculated from [LiDAR](https://en.wikipedia.org/wiki/Lidar) or some other accurate method, and our goal is to get as close as possible to those values using only the images above. 

## A naive approach

So let's try and calculate disparity for the images above.   
There are many, many approaches to calculating disparity, but let us begin with the most simple approach we can think of.   
As a start, let us go through each row of pixels in the left image, and for that pixel, try and find the most similar pixel in the right image. 

So let us try and take the squared difference between pixels values as our similarity metric.
As we are going to be doing the same thing for every row of pixels, we are just going to define a function that does the basic logic, and then apply the same function to every case. 

In [ ]:
function smallest_diff(left_pixel, row, metric)

    disparity_candidates = metric.(left_pixel, row)

    # Minus one as Julia counts from 1
    argmin(disparity_candidates) -1
    
end

Let's define a distance metric as the squared distance:

In [ ]:
squared_difference = (x, y) -> (x-y)^2

And as a test case let's create the cartoon image we had above (inverted, for clarity): 

In [ ]:
left_image = zeros(Float64, (8, 8))
right_image = zeros(Float64, (8, 8))
disparity = zeros(UInt8, (8, 8))

left_image[2, 1] = 1
left_image[3, 4:8] = [1 1 1 1 1]

right_image[7, 1] = 1
right_image[4, 4] = 1
right_image[5, 5] = 1
right_image[6, 6] = 1
right_image[7, 7] = 1
right_image[8, 8] = 1


In [ ]:
Gray.(left_image')

In [ ]:
Gray.(right_image')

In [ ]:
for i in range(1, size(left_image)[1])
    for j in range(1, size(left_image)[2])
        disparity[j, i] = max(smallest_diff(left_image[j, i], right_image[j:end, i], squared_difference), 1)
    end
end
    

So how did we do? 

In [ ]:
Gray.(disparity'/5)

So the toy example works! We'll start with the example case above, but for simplicity we'll stick to grayscale at first: 

In [ ]:
using Images, FileIO

tsukuba_left_gray = Gray.(load("../images/dispairity_block_julia/im3.png"))
tsukuba_right_gray = Gray.(load("../images/dispairity_block_julia/im4.png"))
 

tsukuba_left_gray

In [ ]:
function pixel_match(left_image, right_image, metric, max_disp)

    num_rows = size(left_image)[1]
    num_cols = size(left_image)[2]

    disparity = zeros(size(left_image))
    for i in range(1, num_rows)
        for j in range(1, num_cols)
            terminator = min(j+max_disp, num_cols)
            disparity[i, j] = max(smallest_diff(left_image[i, j], right_image[i, j:terminator], metric), 1)
        end
    end

    disparity
end

In [ ]:
tsukuba_disparity_gray = pixel_match(tsukuba_left_gray, tsukuba_right_gray,  squared_difference, 50);

So let's see how we did?

In [ ]:
Gray.(tsukuba_disparity_gray / maximum(tsukuba_disparity_gray)   )

Looking at the predicted disparity, we can see there is some vague resemblance to the input image, but we're still pretty far from the target: ![https://vision.middlebury.edu/stereo/eval/newEval/tsukuba/](../images/dispairity_block_julia/groundtruth.png) 


As you can imagine, we are only comparing single channel pixels values, and it's very likely that we might just find a better match by chance. In grayscale we are only matching pixel intensity, and we have no idea whether something is bright green, or bright red. 

So let's try and improve the odds of a good match by adding colour. 

In [ ]:
tsukuba_left_rgb = load("../images/dispairity_block_julia/im3.png")
tsukuba_right_rgb = load("../images/dispairity_block_julia/im4.png")
 

tsukuba_left_rgb




In [ ]:
squared_difference_rgb = (x, y) -> ((x.b-y.b)^2 + (x.g-y.g)^2 + (x.r-y.r)^2)

In [ ]:
pix = tsukuba_left_rgb[150, 200] - tsukuba_right_rgb[150, 200]
print(Gray(pix))

In [ ]:
squared_difference_rgb(tsukuba_left_rgb[150, 200], tsukuba_right_rgb[150, 200])

In [ ]:
tsukuba_disparity_rgb = pixel_match(tsukuba_left_rgb, tsukuba_right_rgb,  squared_difference_rgb, 50);

In [ ]:
Gray.(tsukuba_disparity_rgb / maximum(tsukuba_disparity_rgb))

In [ ]:
asdasd 


asdasd

So, not much of an improvement.  

Can we do better? 

## Block matching

The obvious downside of the naive approach above is that it only ever looks at one pixel (in each image) at a time.  
That's not a lot of information, and also not how we intuitively match objects.   

Look at the image below. Can you guess the best match for the pixel in the row of pixels below it? 

![Own work](../images/dispairity_block_julia/pixel_match.png)


Given only this information, it's impossible for us to guess whether the green pixel matches with the pixels at location 3, 5 or 7.  

If however I was to give you more context, i.e. a block of say 3x3 pixels, would this make things simpler? 

![Own work](../images/dispairity_block_julia/block_match.png)

In this case, there is an unambiguous answer, which is the principle behind block-matching. 

We can modify the function `pixel_match` above to instead match blocks of pixels. 

In [ ]:
sum_squared_difference_rgb_block = (x, y) -> sum((channelview(x) - channelview(y)).^2)

In [ ]:
function smallest_block_diff(left_block, row, metric, block_size)

    view_length = size(row)[2]
    disparity_candidates = zeros(view_length - 2*block_size)

    for i in range(block_size + 1, view_length - block_size )
        
        disparity_candidates[i-block_size] = metric(left_block, row[:, i-block_size:i+block_size])
    end
    limit = block_size^2 + block_size
    # Minus one as Julia counts from 1
    
    minval = minimum(disparity_candidates)
    if  minval < limit
        return argmin(disparity_candidates) -1
    else
        return 0
    end
end

In [ ]:
#

In [ ]:
function block_match(left_image, right_image, block_size, metric, max_disp)

    
    num_rows = size(left_image)[1]
    num_cols = size(left_image)[2]

    disparity = zeros(size(left_image))
    for i in range(block_size + 1, num_rows - block_size)
        for j in range(block_size + 1, num_cols - block_size)
            terminator = min(j+max_disp, num_cols)

            row_block_index = i-block_size:i+block_size
            col_block_index = j-block_size:j+block_size
            
            disparity[i, j] = max(smallest_block_diff(left_image[row_block_index, col_block_index], right_image[row_block_index, j-block_size:terminator], metric, block_size), 1)
        end
    end

    disparity
end

In [ ]:
tsukuba_disparity_rgb = block_match(tsukuba_right_rgb, tsukuba_left_rgb, 5,  sum_squared_difference_rgb_block, 30);

In [ ]:
Gray.(tsukuba_disparity_rgb / (maximum(tsukuba_disparity_rgb)))

In [ ]:
maximum(tsukuba_disparity_rgb)

In [ ]:
function smallest_block_diff(left_block, row, metric, block_size)

    view_length = size(row)[2]
    disparity_candidates = zeros(view_length - 2*block_size)

    for i in range(block_size + 1, view_length - block_size )
        
        disparity_candidates[i-block_size] = metric(left_block, row[:, i-block_size:i+block_size])
    end

    return disparity_candidates

end

In [ ]:
block_size = 5
i = 100
j = 220
x = tsukuba_left_rgb[i-block_size:i+block_size, j-block_size:end]#
y = tsukuba_right_rgb[i-block_size:i+block_size, j-block_size:j+block_size]#end]

In [ ]:
x

In [ ]:
diffs = smallest_block_diff(y, x, sum_squared_difference_rgb_block, 5)

In [ ]:
plot(diffs)

In [ ]:
160 - argmin(diffs)

In [ ]:
minimum(diffs)

In [ ]:
x[:, index-block_size:index+block_size] - y

In [ ]:
index = 21

In [ ]:
y

In [ ]:
x[:, index-block_size:index+block_size]

In [ ]:
channelview(x[:, index-block_size:index+block_size]) #- 

In [ ]:
channelview(y)

In [ ]:
channelview(x[:, index-block_size:index+block_size]) - channelview(y)

In [ ]:
sum_squared_difference_rgb_block(x[:, index-block_size:index+block_size], y)

In [ ]:
x = i-block_size:i+block_size

In [ ]:
img_path = "images/groundtruth.png"
img = load(img_path)